# TITANIC

** this notebook assumes that you came from the report on the link below :**

** https://khaledsalahblog.wordpress.com/titanic/**


** if not , you should visit the blog first. **

**all the data sets and code files could be found on this link :             https://www.dropbox.com/sh/6ngfynmla5thndv/AABI_aQIO1VHsqWG0L5MdALKa?dl=0  **



In [432]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

titanic = pd.read_csv("./titanic-data.csv",header=0)


# 1. CLEANING DATA

##  we will search for missing/unreliable data , make a new data set with cleaned data, and make it possible for me to print out the faulty data 


 I will check the important fields i'll need for my analysis which are ['Survived', 'Sex', 'Age', 'Pclass']
 and make a new data frame for the cleaned data

>titanic_cleaned = titanic.apply(clean_data, axis=1)

 i'll apply the function clean_data  and return a data frame includes all the rows 
   that do NOT have any missing/unreliable fields.


In [433]:
# a data frame that will contains the cleaned data
titanic_cleaned=pd.DataFrame(columns=titanic.columns)

# a data frame that will contains the  rows that contains missing/unreliable data
titanic_unreliable_rows = pd.DataFrame(columns=titanic.columns)

def clean_data(person):
    #now for each person I'll check the 4 columns : ['Survived', 'Sex', 'Age', 'Pclass'] and if any of them\
    #  has unwanted data , i'll return None , otherwise i'll return the row
    #  and i'll add the unreliable row to the df titanic_unreliable_rows
    
    if  person['Survived'] not in [0,1] or \
        person['Sex'] not in ['male','female'] or \
        np.isnan(person['Age']) or \
        person['Pclass'] not in [1,2,3] :
            
            titanic_unreliable_rows.loc[len(titanic_unreliable_rows)] = person
            return None
      
    return person # it will reach this line if all the if conditions are false then the line is good

#apply the function to each row
titanic_cleaned=titanic.apply(clean_data, axis=1) 

#drop all the None rows in place
titanic_cleaned.dropna(axis=0,how='all',inplace=True)

# uncomment the next comment to view the new data set
#titanic_cleaned

### The function below print out the faulty rows to the std output :

In [434]:
# this function prints some information about the dataframe we made "titanic_unreliable_rows" 
# it has on optional argument "Columns" which is a list of one or more of the columns [Age,Pclass,Sex,Survived].
#     if the argument has on or more of this values it will print out it's unrelieable rows.
#     and if not , this function will only print the number of faulty rows for each column .
def titanic_cleaning_INFO(columns=None):
    
    #make new 4 dataframes contains  the faulty data for each columns .
    faulty_age      = titanic_unreliable_rows[  np.isnan(titanic_unreliable_rows['Age']) ]
    faulty_pclass   = titanic_unreliable_rows[  ~titanic_unreliable_rows['Pclass'].isin([1,2,3]) ]
    faulty_sex      = titanic_unreliable_rows[  ~titanic_unreliable_rows['Sex'].isin(['male','female']) ]
    faulty_survived = titanic_unreliable_rows[  ~titanic_unreliable_rows['Survived'].isin([0,1]) ]
        
    # print out the number of rows in each of them     
    print \
"number of rows that has unreliable/missing Age values are : {0}, \n\
number of rows that has unreliable/missing Pclass values are : {1}, \n\
number of rows that has unreliable/missing Sex values are : {2}, \n\
number of rows that has unreliable/missing Survived values are : {3}. \n" \
            .format(len(faulty_age),\
                   len(faulty_pclass),
                   len(faulty_sex),
                   len(faulty_survived) )

    
    
    
    # if columns != None , print the df corresponds to argument
    if columns :
        # if columns is one variable "string" then make it a list so I can iterate on it .
        if type(columns) != list :  columns = [columns] 
            
        for column in columns :
            if column   == 'Age'      :   print faulty_age
            elif column == 'Pclass'   :   print faulty_pclass
            elif column == 'Sex'      :   print faulty_sex
            elif column == 'Survived' :   print faulty_survived
            else : 
                print "wrong argument : {0}\n".format(column) #for wrong data

                
#now it's time to use it .
titanic_cleaning_INFO() 

number of rows that has unreliable/missing Age values are : 177, 
number of rows that has unreliable/missing Pclass values are : 0, 
number of rows that has unreliable/missing Sex values are : 0, 
number of rows that has unreliable/missing Survived values are : 0. 



### Save the data locally so I can import it to excel/google-sheets and analyse it

In [435]:
#change it to True to execute
if False :
    titanic_cleaned.to_csv("./titanic_cleaned.csv",index=False)

# ----------------------------------------------------------------------------------------

# 2. TESTING VARIABLES

## I'll test the three variables "age","sex","Pclass" as mentioned in the report.

In [436]:
# first i'll write a function that will help me in testing the variables
# the function will take a row with two columns as input [our variable, Survived ] , and will return 1 of it meets the assumption and 0 O.W .
# then this function will be called with S.apply()

def test_assum(S) :
    #index 'Survived' should be always there, and S[1] should'nt be none
    if not 'Survived' in S.index :
        return -999
    if S[1] == None : return 0
    
    #the variable "SEX"'s part
    if 'Sex' in S.index :
        if ( S['Sex'] == 'male' and not S['Survived'] ) or ( S['Sex'] == 'female' and S['Survived'] ) :
            return 1
        else:
            return 0
    #*******************************
    
    #the variable "AGE"'s part
    elif 'Age' in S.index :
        if ( S['Age'] <= 16 and S['Survived'] ) or ( S['Age'] >= 60 and S['Survived']) :
            return 1
        elif not S['Survived'] :
            return 1
        else :
            return 0
    #***************************************
    
    #the variable "Pclass"'s part
    #here i'll calculate for cass 1 and class 3 only as class 2 has probability .5 , class 1 has 1 , class 3 has 0.        
    elif 'Pclass' in S.index :
        if ( S['Pclass'] == 1 and S['Survived'] ) or ( S['Pclass'] == 3 and not S['Survived'] ):
            return 1
        else :
            return 0
    #*********************************************************


#### Now I will make pair of columns for each variable .
####      titanic_cleaned[[ variable , 'Survived'] ]
#### then we will apply the above function on each of them in separate cell , and get the output.

In [437]:
# the variable "AGE" 

age_T = titanic_cleaned[['Survived','Age']].apply(test_assum, axis=1).sum(skipna=1)
age_F = titanic_cleaned['Age'].count()-age_T
age_p = (float(age_T)/(age_T+age_F))*100

print "True age assumptions : {0}, False age assumptions : {1}\nAge is effective by : {2:.02f} %"\
                                                                     .format(age_T,age_F,age_p)

True age assumptions : 486, False age assumptions : 228
Age is effective by : 68.07 %


In [438]:
#the variable "SEX"

sex_T = titanic_cleaned[['Survived','Sex']].apply(test_assum, axis=1).sum(skipna=1)
sex_F = titanic_cleaned['Sex'].count()-sex_T
sex_p = (float(sex_T)/(sex_T+sex_F))*100
print "True sex assumptions : {0}, False sex assumptions : {1}\nGender is effective by : {2:.02f} %"\
                                                                        .format(sex_T,sex_F,sex_p)

True sex assumptions : 557, False sex assumptions : 157
Gender is effective by : 78.01 %


In [439]:
#the variable "Pclass"

class_T = titanic_cleaned[['Survived','Pclass']].apply(test_assum, axis=1).sum(skipna=1)
class_F = titanic_cleaned[titanic_cleaned['Pclass'] != 2 ]['Pclass'].count()-class_T
class_p = (float(class_T)/(class_T+class_F))*100
print "True Pclass assumptions : {0}, False Pclass assumptions : {1}\nPassenger's class is effective by : {2:.02f} %"\
                                                                        .format(class_T,class_F,class_p)

True Pclass assumptions : 392, False Pclass assumptions : 149
Passenger's class is effective by : 72.46 %


# ----------------------------------------------------------------------------------------

# 3. LIKELY SURVIVED

#### add a new column to our data-set "titanic_cleaned" called "LS" [likely survived] which contains the percentage of every passenger of how likely he should survive

In [440]:
# make a list to hold the values of the new columns , it's index will be the passengers' IDs
LS= pd.Series(index=titanic_cleaned['PassengerId'])

In [441]:
#here I will define a function takes a passenger "row of the data set" and return it's P value .

def calculate_P(passenger):
    AF =  31.15
    SF =  35.7 
    CF =  33.16

    if  60 > passenger['Age'] > 16 :
        A = 0
    else :
        A = 1
    
    if passenger['Sex'] == 'female' :
        S = 1
    else :
        S = 0
    
    if passenger['Pclass'] == 1 :
        C = 1
    elif passenger['Pclass'] == 2 :
        C = .5
    else :
        C = 0
    
    P = A*AF + S*SF +  C*CF
    
    return  P


In [442]:
# after we wrote the function .. we will apply it to each row in the data set

LS=titanic_cleaned.apply(calculate_P, axis=1)

#uncomment the below line to print out the series
#LS

In [443]:
# the last step ,  we will add the series into the data-set as a column

titanic_cleaned['LS'] = LS

In [444]:
# And save the data set to a CSV file.
#change it to True to execute
if False :
    titanic_cleaned.to_csv("./titanic_cleaned.csv",index=False)

# ----------------------------------------------------------------------------------------

# 4. RESULTS' CHECK

### I'll test the results of the new column and compare it to the column 'survived' and calculate in percentages how this data is accurate .

In [445]:
# calculate how many True data which is how many passengers with lower than 50%LS is dead plus how many passengers \
#     with higher than 50%LS is survived

#this function would help me.
#it takes a series of two elements ['Survived','LS'] and return 0 or 1 .
#this function works the same way as the function test_assum

def test_results(S):
    if  'LS' not in S.index or 'Survived' not in S.index :
        return -999 #return error
    
    if S['LS'] >= 50 and S['Survived'] == 1 :
        return 1
    elif S['LS'] < 50 and S['Survived'] == 0 :
        return 1
    else :
        return 0
    
    

In [446]:
#now I call the function and send to it a series of the two columns

results_True = titanic_cleaned[['Survived','LS']].apply(test_results, axis=1).sum(skipna=1)
results_False = titanic_cleaned['LS'].count() - results_True
results_perc =(float(results_True)/titanic_cleaned['LS'].count())*100
print "there are {0} True results, and {1} False results \n The prediction was accurate by : {2:.02f}% "\
                                        .format(results_True,results_False,results_perc)

there are 562 True results, and 152 False results 
 The prediction was accurate by : 78.71% 


# ----------------------------------------------------------------------------------------

# 5. FILL THE MISSING AGE VALUES

### First lets produce some statistics about the relation between Age and 'gender, Survival and pclass'

In [447]:
#1. the correlation between them 
titanic_cleaned[['Age','Survived','Sex','Pclass']].corr()

,Age,Survived,Pclass
Age,1.000000,-0.077221,-0.369226
Survived,-0.077221,1.000000,-0.359653
Pclass,-0.369226,-0.359653,1.000000


In [448]:
#2. statistics between Age and each of male and female .
print "Statistics For males' Age : "
print titanic_cleaned[titanic_cleaned['Sex']=='male']['Age'].describe()
print "\nStatistics For females' Age : "
print titanic_cleaned[titanic_cleaned['Sex']=='female']['Age'].describe()

Statistics For males' Age : 
count    453.000000
mean      30.726645
std       14.678201
min        0.420000
25%       21.000000
50%       29.000000
75%       39.000000
max       80.000000
Name: Age, dtype: float64

Statistics For females' Age : 
count    261.000000
mean      27.915709
std       14.110146
min        0.750000
25%       18.000000
50%       27.000000
75%       37.000000
max       63.000000
Name: Age, dtype: float64


In [449]:
#3. statistics between Pclasses and Age
print "Statistics For 1st class's Age : "
print titanic_cleaned[titanic_cleaned['Pclass']== 1]['Age'].describe()

print "\nStatistics For 2nd class's Age : "
print titanic_cleaned[titanic_cleaned['Pclass']== 2]['Age'].describe()

print "\nStatistics For 3rd class's Age : "
print titanic_cleaned[titanic_cleaned['Pclass']== 3]['Age'].describe()

Statistics For 1st class's Age : 
count    186.000000
mean      38.233441
std       14.802856
min        0.920000
25%       27.000000
50%       37.000000
75%       49.000000
max       80.000000
Name: Age, dtype: float64

Statistics For 2nd class's Age : 
count    173.000000
mean      29.877630
std       14.001077
min        0.670000
25%       23.000000
50%       29.000000
75%       36.000000
max       70.000000
Name: Age, dtype: float64

Statistics For 3rd class's Age : 
count    355.000000
mean      25.140620
std       12.495398
min        0.420000
25%       18.000000
50%       24.000000
75%       32.000000
max       74.000000
Name: Age, dtype: float64


In [450]:
#4.statistics between Survival and age

print "Statistics For passengers Survived's Age : "
print titanic_cleaned[titanic_cleaned['Survived']== 1]['Age'].describe()


print "Statistics For passengers didn't survive's  Age : "
print titanic_cleaned[titanic_cleaned['Survived']== 0]['Age'].describe()

Statistics For passengers Survived's Age : 
count    290.000000
mean      28.343690
std       14.950952
min        0.420000
25%       19.000000
50%       28.000000
75%       36.000000
max       80.000000
Name: Age, dtype: float64
Statistics For passengers didn't survive's  Age : 
count    424.000000
mean      30.626179
std       14.172110
min        1.000000
25%       21.000000
50%       28.000000
75%       39.000000
max       74.000000
Name: Age, dtype: float64


----------------------------

### Now I'll fill the missing age of a passenger by calculation the average of his "Sex,Survived and pclass" columns


In [451]:
# first make variables hold the three averages.
male_avg = titanic_cleaned[titanic_cleaned['Sex']=='male']['Age'].mean()
female_avg = titanic_cleaned[titanic_cleaned['Sex']=='female']['Age'].mean() 

class1_avg = titanic_cleaned[titanic_cleaned['Pclass']== 1]['Age'].mean()
class2_avg = titanic_cleaned[titanic_cleaned['Pclass']== 2]['Age'].mean()
class3_avg = titanic_cleaned[titanic_cleaned['Pclass']== 3]['Age'].mean()

survived_avg =  titanic_cleaned[titanic_cleaned['Survived']== 1]['Age'].mean()
not_survived_avg = titanic_cleaned[titanic_cleaned['Survived']== 0]['Age'].mean()



In [452]:
# a new data set to hold "titanic" with filled missing data

titanic_filled = titanic

In [453]:
# make a function that takes a person if his age is not a number return without effect ,
# otherwise, fix his age and return the new series.
# then it will be called using apply.

def fill_missings(person):

    if not np.isnan(person['Age']) : return person
    # set variables.
    if person['Sex'] == 'male' :
        sex = male_avg
    else :
        sex = female_avg
        
    if person['Survived']:
        survived = survived_avg
    else :
        survived = not_survived_avg
        
    if person['Pclass']  == 1 :
        pclass = class1_avg
    elif person ['Pclass'] == 2 :
        pclass = class2_avg
    else :
        pclass = class3_avg
    
    # Now calculate the age  :
    person['Age'] = (sex + survived + pclass)/4.0
    
    return person

In [454]:
# Now apply the function to all passengers :

titanic_filled = titanic_filled.apply(fill_missings, axis=1)

In [455]:
# And save the data set to a CSV file.
#change it to True to execute
if False :
    titanic_filled.to_csv("./titanic_filled.csv",index=False)

----